This file allows for testing the GraphWorld setup with GNN implementations.
It is currently set up to test the SSL methods for the JL benchmarker.

Through this notebook you can attach a debugger.
Note that graph_tool does not work on windows, so we cannot use the graph generators.
Instead, we use the standard datasets from PyG.

In [1]:
import os
import sys
module_path = os.path.abspath(os.path.join('..'))
if module_path not in sys.path:
    sys.path.append(module_path)

from graph_world.self_supervised_learning.benchmarker_jl import NNNodeBenchmarkerJL
from graph_world.models.basic_gnn import GCN, SuperGAT
from torch_geometric.datasets import Planetoid

from graph_world.self_supervised_learning.pretext_tasks.contrastive_based import *


In [6]:
# Parameter setup (for cora)
benchmark_params = {
    'epochs' : 500,
    'lr' : 0.01,
    'lambda' : 1
}

h_params = {
    'in_channels' : 1433,
    'hidden_channels' : 16,
    'num_layers' : 2,
    'dropout' : 0.5,
    'edge_mask_ratio' : 0.2,
    'feature_mask_ratio' : 0.3
}

generator_config = {
    'num_clusters' : 7,
}

pretext_task = GBT

In [3]:
# Get dataset
dataset = Planetoid(root='/tmp/Cora', name='Cora')[0]

In [7]:
# Training. You can attach a debugger to w/e is needed inside train
benchmarker = NNNodeBenchmarkerJL(generator_config=generator_config, model_class=GCN, 
                benchmark_params=benchmark_params, h_params=h_params, pretext_task=pretext_task)
benchmarker.SetMasks(train_mask=dataset.train_mask, val_mask=dataset.val_mask, test_mask=dataset.test_mask)
benchmarker.train(data=dataset, tuning_metric="rocauc_ovr", tuning_metric_is_loss=False)

GCN(1433, 16, num_layers=2)
tensor(13.5242, grad_fn=<AddBackward0>)
tensor(13.0090, grad_fn=<AddBackward0>)


c:\Users\bertr\miniconda3\envs\thesis\lib\site-packages\torch_geometric\deprecation.py:12: UserWarning: 'dropout_adj' is deprecated, use 'dropout_edge' instead
  warnings.warn(out)
c:\Users\bertr\miniconda3\envs\thesis\lib\site-packages\torch_geometric\deprecation.py:12: UserWarning: 'dropout_adj' is deprecated, use 'dropout_edge' instead
  warnings.warn(out)
c:\Users\bertr\miniconda3\envs\thesis\lib\site-packages\torch_geometric\deprecation.py:12: UserWarning: 'dropout_adj' is deprecated, use 'dropout_edge' instead
  warnings.warn(out)


tensor(10.6090, grad_fn=<AddBackward0>)
tensor(9.0832, grad_fn=<AddBackward0>)
tensor(9.4364, grad_fn=<AddBackward0>)


c:\Users\bertr\miniconda3\envs\thesis\lib\site-packages\torch_geometric\deprecation.py:12: UserWarning: 'dropout_adj' is deprecated, use 'dropout_edge' instead
  warnings.warn(out)
c:\Users\bertr\miniconda3\envs\thesis\lib\site-packages\torch_geometric\deprecation.py:12: UserWarning: 'dropout_adj' is deprecated, use 'dropout_edge' instead
  warnings.warn(out)
c:\Users\bertr\miniconda3\envs\thesis\lib\site-packages\torch_geometric\deprecation.py:12: UserWarning: 'dropout_adj' is deprecated, use 'dropout_edge' instead
  warnings.warn(out)
c:\Users\bertr\miniconda3\envs\thesis\lib\site-packages\torch_geometric\deprecation.py:12: UserWarning: 'dropout_adj' is deprecated, use 'dropout_edge' instead
  warnings.warn(out)
c:\Users\bertr\miniconda3\envs\thesis\lib\site-packages\torch_geometric\deprecation.py:12: UserWarning: 'dropout_adj' is deprecated, use 'dropout_edge' instead
  warnings.warn(out)


tensor(8.1568, grad_fn=<AddBackward0>)
tensor(7.9933, grad_fn=<AddBackward0>)
tensor(7.4111, grad_fn=<AddBackward0>)


...

tensor(3.4491, grad_fn=<AddBackward0>)
tensor(3.5136, grad_fn=<AddBackward0>)
tensor(3.5126, grad_fn=<AddBackward0>)


c:\Users\bertr\miniconda3\envs\thesis\lib\site-packages\torch_geometric\deprecation.py:12: UserWarning: 'dropout_adj' is deprecated, use 'dropout_edge' instead
  warnings.warn(out)
c:\Users\bertr\miniconda3\envs\thesis\lib\site-packages\torch_geometric\deprecation.py:12: UserWarning: 'dropout_adj' is deprecated, use 'dropout_edge' instead
  warnings.warn(out)
c:\Users\bertr\miniconda3\envs\thesis\lib\site-packages\torch_geometric\deprecation.py:12: UserWarning: 'dropout_adj' is deprecated, use 'dropout_edge' instead
  warnings.warn(out)


tensor(3.5438, grad_fn=<AddBackward0>)
tensor(3.5053, grad_fn=<AddBackward0>)
tensor(3.5130, grad_fn=<AddBackward0>)
tensor(3.5688, grad_fn=<AddBackward0>)


c:\Users\bertr\miniconda3\envs\thesis\lib\site-packages\torch_geometric\deprecation.py:12: UserWarning: 'dropout_adj' is deprecated, use 'dropout_edge' instead
  warnings.warn(out)
c:\Users\bertr\miniconda3\envs\thesis\lib\site-packages\torch_geometric\deprecation.py:12: UserWarning: 'dropout_adj' is deprecated, use 'dropout_edge' instead
  warnings.warn(out)


([15.486917495727539,
  14.96762466430664,
  12.559062004089355,
  11.02665901184082,
  11.373835563659668,
  10.087130546569824,
  9.914985656738281,
  9.329959869384766,
  9.234305381774902,
  8.682376861572266,
  8.511859893798828,
  8.26390266418457,
  8.411825180053711,
  8.079158782958984,
  7.625940322875977,
  7.316616058349609,
  7.507457733154297,
  7.494594573974609,
  7.163559436798096,
  7.041815757751465,
  6.889572620391846,
  6.8301568031311035,
  6.633142471313477,
  6.7747602462768555,
  6.845067024230957,
  6.709259033203125,
  6.524022579193115,
  6.54057502746582,
  6.449981689453125,
  6.477496147155762,
  6.505004405975342,
  6.4656596183776855,
  6.46336555480957,
  6.558968544006348,
  6.5407915115356445,
  6.553307056427002,
  6.33376407623291,
  6.424378395080566,
  6.521533966064453,
  6.382404327392578,
  6.454660892486572,
  6.3318376541137695,
  6.236243724822998,
  6.286423683166504,
  6.149539470672607,
  6.155597686767578,
  6.113874435424805,
  6.0327